# Продолжаем говорить про тесты

Мы начинали говорить про тестирование в прошлый раз, рассмотрели одну из простых классификаций тестов:

![Пирамида](https://www.vendure.io/blog/2021/03/whats-up-with-e2e-tests/testing-pyramid.png)

И посмотрели, как в простом варианте писать unit-тесты. И работать с `pytest`.

# Обзор инструментов

Пронесемся по верхам, чтобы было общее понимание, что это вообще такое, как примерно использовать
и куда лезть, чтобы погрузиться глубже.

## `unittest.mock.` (https://docs.python.org/3/library/unittest.mock.html)

### Проблема: ходим в базу (или делаем что-нибудь тяжелое / влияющее на внешний мир) -- как это тестировать?

In [36]:
from time import sleep


class DatabaseError(Exception):
    ...


def get_data_from_database():
    """ Некоторая функция, которая производит "тяжелые операции", например ходит в базу
    или еще что-то делает, что в тестах как-то не хочется делать.
    """
    sleep(10)  # имитируем тяжелую работу
    return [1, 2, 3]  # и выходные данные


def process_data_from_database():
    """ А эту функцию мы хотим протестировать.
    """
    try:
        data = get_data_from_database()
    except DatabaseError:
        print("Get DB error")
        return
    print(f"Received data: {data}")
    ... # processing
    return data

### Решение: `Mock` и `patch`

In [3]:
from unittest.mock import Mock

Mock and MagicMock objects create all attributes and methods as you access them and store details of how they have been used. You can configure them, to specify return values or limit what attributes are available, and then make assertions about how they have been used.

In [4]:
m = Mock()

In [12]:
m.a(1)

<Mock name='mock.a()' id='140188080280048'>

In [32]:
m.a.b.c.d

<Mock name='mock.a.b.c.d().b()' id='140188122122080'>

In [14]:
m.return_value = 100500
m(1, 2, a=3), m.call_count, m.call_args_list

(100500, 1, [call(1, 2, a=3)])

In [15]:
m.side_effect = Exception("OMG!")
m()

Exception: OMG!

In [16]:
m.side_effect = [1, KeyError, 3]

In [25]:
m.call_args_list

[]

In [23]:
m.reset_mock()

In [26]:
m = Mock()

In [27]:
m.return_value = KeyError("123")

In [30]:
m()

KeyError('123')

---

In [33]:
from unittest.mock import patch

In [34]:
patch?

In [41]:
with patch("__main__.get_data_from_database") as m:
    data = [4, 5, 6]
    m.return_value = data
    assert process_data_from_database() == data

Received data: [4, 5, 6]


In [42]:
with patch("__main__.get_data_from_database") as m:
    m.side_effect = DatabaseError
    assert process_data_from_database() is None

Get DB error


In [44]:
with patch("__main__.get_data_from_database"):
    get_data_from_database()
get_data_from_database()

KeyboardInterrupt: 

In [48]:
@patch("__main__.process_data_from_database")
@patch("__main__.get_data_from_database")
def test_something(m_get_data, m_process):
    m_get_data.return_value = "Get data"
    m_process.return_value = "Processing"
    print(get_data_from_database())
    print(process_data_from_database())


test_something()

Get data
Processing


Почитайте, плз, про `mocker`

In [ ]:
def test_something(mocker):
    ...

Вопрос:

```python
patch("__main__.Foo.bar") # или все-таки `patch.object(Foo, "bar")`
```

Ответ: все-таки `patch.object(Foo, "bar")` из-за особенностей импорта.

In [49]:
new_bar = lambda self, a, b, c: 100500

class Foo:
    def bar(self):
        pass


with patch.object(Foo, "bar", new=new_bar) as m:
    assert Foo().bar(1, 2, c=3) == 100500

In [53]:
foo = {}
with patch.dict(foo):
    foo[1] = 2
    assert foo == {1: 2}
assert foo == {}

In [54]:
import os
os.environ

environ{'TERM_SESSION_ID': 'w1t0p2:1D4E21F8-1E49-466F-86DF-036A1DFD4319',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.gutxXDqXon/Listeners',
        'LC_TERMINAL_VERSION': '3.4.8',
        'COLORFGBG': '15;0',
        'ITERM_PROFILE': 'Default',
        'XPC_FLAGS': '0x0',
        'LANG': 'en_US.UTF-8',
        'PWD': '/Users/iporyadkov/Desktop/PYCHARM/OZON.MASTERS/gitlab/day_04',
        'SHELL': '/bin/zsh',
        '__CFBundleIdentifier': 'com.googlecode.iterm2',
        'SECURITYSESSIONID': '186a8',
        'TERM_PROGRAM_VERSION': '3.4.8',
        'TERM_PROGRAM': 'iTerm.app',
        'PATH': '/Users/iporyadkov/.cabal/bin:/Users/iporyadkov/.ghcup/bin:/Users/iporyadkov/anaconda2/envs/py38/bin:/Users/iporyadkov/anaconda2/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/Users/iporyadkov/Library/Python/2.7/bin:/Users/iporyadkov/anaconda2/bin:/usr/local/Cellar/postgresql@11/11.8_3/bin:~/.bin:/Users/iporyadkov/.kafka/kafka_2.13-2.6.0/bin:/Users/iporyadk

### Мокать или нет?

Все подряд мокать не нужно -- ту же базу лучше забить тестовыми данными, она и в других тестах пригодится \+ тестирование будет более полным (это уже будет интеграционный тест).

Сложные сервисы, хранящие состояние, иногда приходится мокать целиком (писать их небольшую версию), а отслеживать вызовы, аргументы и возвращаемые значения можно вот так, например: https://github.com/pytest-dev/pytest-mock/#spy

### `MagicMock`

Magic methods:
- `str(x)` -> `x.__str__()`
- `repr(x)` -> `x.__repr__()`
- `a + b` -> `a.__add__(b)` (или `b.__radd__(a)`)
- `next(x)` -> `x.__next__()`
- `iter(x)` -> `x.__iter__()`
- `x[i]` -> `x.__getitem__(i)`
- ... и другие https://docs.python.org/3/library/unittest.mock.html#mocking-magic-methods

In [55]:
m = Mock()

In [57]:
m[0]  # m.__getitem__(0)

TypeError: 'Mock' object is not subscriptable

In [58]:
from unittest.mock import MagicMock

In [69]:
mm = MagicMock()

In [70]:
mm + mm

<MagicMock name='mock.__add__()' id='140188083744432'>

In [71]:
mm.__add__.assert_called_once_with(mm)

In [74]:
with mm() as x:
    print(x)

<MagicMock name='mock().__enter__()' id='140188102089072'>


In [80]:
mm().__exit__.assert_called()

### `Any`

In [85]:
from unittest.mock import ANY

In [81]:
m = Mock()

In [82]:
m(1, 2, c=3)

<Mock name='mock()' id='140188103773488'>

In [87]:
m.assert_called_with(1, ANY, c=ANY)

## `pytest.fixture` (https://docs.pytest.org/en/6.2.x/fixture.html)

См. 'testing/test_fixtures.py' и 'testing/conftest.py'.

Запускайте тесты так:
```bash
pytest -vs
```
или так, чтобы ограничить набор тестов:
```bash
pytest -vs -k <substring>
```

---

# Как писать тесты, чтобы было ~~не стыдно~~ стыдно чуть меньше, чем обычно?

- Следовать парадигме:
    - preparation (фикстуры, моки)
    - action
    - assertion
    - cleanup (фикстуры)
- Один кейс -- один тест
- Искать и тестировать краевые случаи
- Все что происходит в тесте -- остается в тесте
- Не пренебрегайте тестами -- они ваши друзья